# Initialize

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import $ivy.$                                   // Or use any other 2.x version here

import org.apache.log4j.{Level, Logger}

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)

# Create Spark session

In [2]:
import org.apache.spark.sql._
val sparkSession = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
sparkSession.getClass
import sparkSession.sql

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._

sparkSession: SparkSession = org.apache.spark.sql.SparkSession@1a71f75
res1_2: Class[T] = class org.apache.spark.sql.SparkSession
import sparkSession.sql

# Load data

In [29]:
val result = sparkSession.read.format("csv")
      .option("delimiter", "\t")
      .option("header", "true")
      .option("quote", "'")
      .option("nullValue", "NULL")
      .option("inferSchema", "true")
      .load("/home/pi/sparktest/data/privpref")

load at cmd28.sc:7

1 / 1

load at cmd28.sc:7

3 / 3

result: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]

In [30]:
result.registerTempTable("result")
result.printSchema

root
 |-- REF_NUM0: long (nullable = true)
 |-- LAST_USED_DT: timestamp (nullable = true)
 |-- 3: timestamp (nullable = true)
 |-- SOURCE_IDENT_TP_CD: integer (nullable = true)
 |-- NAME4: string (nullable = true)
 |-- X_COUNTRY_TP_CD: integer (nullable = true)
 |-- ISO_CODE: string (nullable = true)
 |-- NAME7: string (nullable = true)
 |-- CONT_ID: long (nullable = true)
 |-- CONT_METH_TP_CD: integer (nullable = true)
 |-- REF_NUM10: string (nullable = true)
 |-- MINSTNUM: long (nullable = true)
 |-- CNT: integer (nullable = true)
 |-- MAXPP: string (nullable = true)
 |-- MINPP: string (nullable = true)
 |-- LOCATION_GROUP_ID: long (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)



# get records with different SP

In [72]:
val differentSP = sql("""
select * from result 
where 
((minPP = 'S' and maxPP = 'P') or (minPP = 'P' and maxPP = 'S'))
""")
differentSP.cache
differentSP.registerTempTable("differentSP")
differentSP.show()

show at cmd71.sc:8

1 / 1

+---------------+--------------------+--------------------+------------------+-------------+---------------+--------+-----+------------------+---------------+--------------+-------------+---+-----+-----+------------------+----+-----+---+
|       REF_NUM0|        LAST_USED_DT|                   3|SOURCE_IDENT_TP_CD|        NAME4|X_COUNTRY_TP_CD|ISO_CODE|NAME7|           CONT_ID|CONT_METH_TP_CD|     REF_NUM10|     MINSTNUM|CNT|MAXPP|MINPP| LOCATION_GROUP_ID|YEAR|MONTH|DAY|
+---------------+--------------------+--------------------+------------------+-------------+---------------+--------+-----+------------------+---------------+--------------+-------------+---+-----+-----+------------------+----+-----+---+
|630881052626909|2020-07-22 20:27:...|2020-07-21 20:27:...|            100005|          IWM|            185|      US| WORK|743404802239030641|              2|    7142503000|  17142503000|  2|    S|    P|720046741219991462|2020|    8|  7|
|161521056084933|2020-07-31 12:55:...|2020-07-30

differentSP: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res71_1: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]

# get records with: one is s/p, other is N

In [73]:
val spn = sql("""
select * from result where 
        ((minPP = 'S' or minPP = 'P') and maxPP = 'N')
        or
        ((maxPP = 'S' or maxPP = 'P') and minPP = 'N')
""")
spn.cache
spn.registerTempTable("spn")
spn.show(5)

show at cmd72.sc:9

1 / 1

+---------------+--------------------+--------------------+------------------+-------+---------------+--------+-----+------------------+---------------+------------+------------+---+-----+-----+------------------+----+-----+---+
|       REF_NUM0|        LAST_USED_DT|                   3|SOURCE_IDENT_TP_CD|  NAME4|X_COUNTRY_TP_CD|ISO_CODE|NAME7|           CONT_ID|CONT_METH_TP_CD|   REF_NUM10|    MINSTNUM|CNT|MAXPP|MINPP| LOCATION_GROUP_ID|YEAR|MONTH|DAY|
+---------------+--------------------+--------------------+------------------+-------+---------------+--------+-----+------------------+---------------+------------+------------+---+-----+-----+------------------+----+-----+---+
|190771087546590| 2020-08-06 05:04:00| 2020-08-05 05:04:00|            100010|DSW SAP|             90|      JP| WORK|855438468796951971|              2|079-295-4111| 81792954111|  2|    P|    N|214854384688367691|2020|    8|  7|
|873161058774027|2020-08-01 00:05:...|2020-07-31 00:05:...|            100016|   SFD

spn: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res72_1: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]

# get only one has s/p

In [74]:
val onlyone = sql("""
select * from result where 
        ((maxpp = 'S' or maxpp = 'P')and minpp is null)
            or
        ((minpp = 'S' or minpp = 'P')and maxpp is null)
""")
onlyone.cache
onlyone.registerTempTable("onlyone")
onlyone.count

count at cmd73.sc:9

3 / 3

count at cmd73.sc:9

1 / 1

onlyone: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res73_1: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res73_3: Long = 275L

# other queries

In [76]:
val onlyone5 = sql("""
select * from onlyone
where day = 5
""")
onlyone5.cache
onlyone5.registerTempTable("onlyone5")
onlyone5.count

val onlyone7 = sql("""
select * from onlyone
where day = 7
""")
onlyone7.cache
onlyone7.registerTempTable("onlyone7")
onlyone7.count

count at cmd75.sc:7

3 / 3

count at cmd75.sc:7

1 / 1

count at cmd75.sc:15

3 / 3

count at cmd75.sc:15

1 / 1

onlyone5: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res75_1: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res75_3: Long = 54L
onlyone7: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res75_5: DataFrame = [REF_NUM0: bigint, LAST_USED_DT: timestamp ... 17 more fields]
res75_7: Long = 150L

## find data existed in two consecutive days

In [77]:
sql("""
select count(*) from onlyone5 t1 inner join onlyone7 t2
on t1.cont_id = t2.cont_id
""").show

run at ThreadPoolExecutor.java:1149

3 / 3

show at cmd76.sc:1

3 / 3

show at cmd76.sc:1

1 / 1

+--------+
|count(1)|
+--------+
|      51|
+--------+



## find data only existed in next day

In [71]:
//use temp table:
sql("""
select t1.cont_id as count_id_5,t2.cont_id as count_id_7 from onlyone5 t1 right outer join onlyone7 t2
on t1.cont_id = t2.cont_id
where t1.cont_id is null
""").count

run at ThreadPoolExecutor.java:1149

3 / 3

count at cmd70.sc:1

3 / 3

count at cmd70.sc:1

1 / 1

res70: Long = 99L

In [79]:
//use total table and filter
val onlyin7 = sql("""
select t1.cont_id,t2.cont_id from 
(
    select * from onlyone where day = 5
) t1
right outer join 
(
    select * from onlyone where day = 7
) t2
on t1.cont_id = t2.cont_id
where t1.cont_id is null
""")
onlyin7.cache
onlyin7.count

run at ThreadPoolExecutor.java:1149

3 / 3

count at cmd78.sc:14

3 / 3

count at cmd78.sc:14

1 / 1

onlyin7: DataFrame = [cont_id: bigint, cont_id: bigint]
res78_1: DataFrame = [cont_id: bigint, cont_id: bigint]
res78_2: Long = 99L

In [80]:
onlyin7.rdd.saveAsTextFile("onlyin7.csv")

runJob at SparkHadoopWriter.scala:78

3 / 3

## find data only existed in previous day

In [67]:
sql("""
select t1.cont_id as count_id_5,t2.cont_id as count_id_7 from onlyone5 t1 left outer join onlyone7 t2
on t1.cont_id = t2.cont_id
where t2.cont_id is null
""").show

run at ThreadPoolExecutor.java:1149

3 / 3

show at cmd66.sc:1

1 / 1

show at cmd66.sc:1

2 / 2

+------------------+----------+
|        count_id_5|count_id_7|
+------------------+----------+
|832548840597243771|      null|
|274934228981171182|      null|
|413842389072174062|      null|
+------------------+----------+

